<a href="https://colab.research.google.com/github/Squirrelcoding/parking-meter/blob/main/Cool_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import pandas as pd

In [43]:
!pip3 install OSMPythonTools
!pip3 install gdal

In [17]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='poopnet-4fb22')

In [18]:
# TODO: Write some code that gets all of the boundaries of a parking lot given its ID or something
from OSMPythonTools.api import Api
api = Api()


# Setting up Earth Engine

In [20]:
# import geemap



# m = geemap.Map()

# m.center_object(roi, zoom=15)
# m.addLayer(clipped_image, vis_params, 'NAIP Mosaic', True)

# m

Map(center=[28.358319486027533, -81.55630061726131], controls=(WidgetControl(options=['position', 'transparent…

In [66]:
import os
import geemap
from osgeo import gdal
from PIL import Image


def tif_to_png(filename: str | os.PathLike):
  pre = filename.split(".")[0]
  tif = gdal.Open(filename)
  arr = tif.ReadAsArray().transpose(1, 2, 0)  # Convert CHW to HWC
  img = Image.fromarray(arr.astype(np.uint8))
  os.remove(filename)
  img.save(pre + '.png')

def download_parking_lot(way_id: int):
  way = api.query(f"way/{way_id}")
  roi = ee.Geometry.Polygon(way.geometry()['coordinates'])
  image_collection = (
      ee.ImageCollection('USDA/NAIP/DOQQ')
      .filterBounds(roi)  # Filter to images intersecting the ROI
      .filterDate('2022-01-01', '2023-12-31') # Filter to a recent time frame
      .sort('system:time_start', False) # Sort by date, newest first
  )

  mosaic = image_collection.mosaic()

  vis_params = {
      'bands': ['R', 'G', 'B'],
      'min': 0,
      'max': 255
  }

  clipped_image = mosaic.clip(roi).unmask()
  geemap.ee_export_image(clipped_image, filename=f'{way_id}.tif', scale=0.10394100844860077, region=roi)

In [69]:
id = 121049764
download_parking_lot(id)
tif_to_png(f"{id}.tif")

Generating URL ...
Please wait ...
Data downloaded to /content/121049764.tif
